In [7]:
# Load relevant packages
from bs4 import BeautifulSoup # For parsing html
import requests # makes http requests

from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool 
import time
import threading
from queue import Queue

import csv

In [2]:
# Specify the URLs
URL_MAIN = "http://www.eixgrandegracia.cat"
URL_STORES = "http://www.eixgrandegracia.cat/web/comercios"

In [3]:
# Functions used for parsing 
def parse(url_1,url_2):
    my_info = [] # Complete info for every store
    for i in range(len(get_soup(URL_STORES).select("div.force-box a"))): # searches the whole range of images that direct you to the stores info url
        new_soup = get_soup(url_1).select("div.force-box a")[i]['href'] # gets the url of the store
        web_directed = "{}{}".format(url_2, new_soup) # combines it with the main websites domain 
        my_info.append(get_info(web_directed)[0]) # gets the info for this store
    my_info_csv = make_csv(my_info) # creates a csv with the info of all the stores
    return my_info_csv

def get_soup(url):
    res = requests.get(url) # gets a webpage <Response [200]>
    return BeautifulSoup(res.text, "lxml") # transforms the html code of the requested url in nice for python format

def get_info(url):
    info_one = [] # get's info from a signle item, e.g. the name of the store
    info_all = [] # combines all info of one store
    for j in range(len(get_soup(url).select("label.col-md-9"))): 
        title = " ".join(str.split(get_soup(url).select("div.col-sm-11 h3")[0].text)) # gets the name of the store in nice format
        info_one.append(" ".join(str.split(get_soup(url).select("label.col-md-9")[j].text))) # gets the rest of the info in nice format
    info_all.append({'Name': title, 'URL': info_one[0], 'Product': info_one[1], 'Telephone': info_one[2], "Adress": info_one[3]}) # creates a dictionary with all the info of the store
    return(info_all) 
                
def make_csv(list_of_dictionaries):
    keys = list_of_dictionaries[0].keys() # gets the keys of the list of dictionaries
    with open('stores_info.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys) # creates an object which operates like a regular writer but maps dictionaries onto output rows.
        dict_writer.writeheader() # writes a row with the field names
        dict_writer.writerows(list_of_dictionaries) # writes all elements in rows

In [25]:
# Trial. Workes. Takes 10 minutes
parse(URL_STORES,URL_MAIN)

In [40]:
# For my knowledge 
def f(x):
    sum = 0
    for x in range(1000):
        sum += x*x
    return sum

t1 = time.time()
p = Pool()
result = p.map(f,range(100000))
p.close()
p.join()
print("Pool took", time.time() - t1)
    
t2 = time.time()
result = []
for x in range(100000):
    result.append(f(x))
print("Serial processing took", time.time() - t2)

Pool took 4.23181414604187
Serial processing took 7.84465479850769


In [41]:
# Trial ThreadPool with limited amount of pages
def parse_trial(url_1,url_2):
    my_info = [] # Complete info for every store
    for i in range(3): # searches the whole range of images that direct you to the stores info url
        new_soup = get_soup(url_1).select("div.force-box a")[i]['href'] # gets the url of the store
        web_directed = "{}{}".format(url_2, new_soup) # combines it with the main websites domain 
        my_info.append(get_info(web_directed)[0]) # gets the info for this store
    my_info_csv = make_csv(my_info) # creates a csv with the info of all the stores
    return my_info_csv

In [47]:
# Trial ThreadPool
with ThreadPoolExecutor(50) as pool:
    results = pool.map(parse_trial, URL_STORES, URL_MAIN)
print(list(results))             

MissingSchema: Invalid URL 'h': No schema supplied. Perhaps you meant http://h?